# MMLU-ProX-Lite-Open Dataset Upload to Elluminate

This notebook loads the MMLU-ProX-Lite-open dataset and uploads it to Elluminate, creating separate collections for each language.

## Setup and Imports

In [7]:
import os
from dotenv import load_dotenv
from elluminate import Client
from datasets import load_dataset
import pandas as pd
from tqdm import tqdm
import nest_asyncio

nest_asyncio.apply()

# Load environment variables
load_dotenv()

print("Environment setup complete")

Environment setup complete


## Initialize Elluminate Client

In [2]:
# Initialize Elluminate client
# Make sure ELLUMINATE_API_KEY and ELLUMINATE_BASE_URL are set in your .env file
client = Client()

print("Elluminate client initialized successfully")

Elluminate client initialized successfully


## Load Dataset

In [3]:
# Load the MMLU-ProX-Lite-open dataset
dataset = load_dataset("jphme/MMLU-ProX-Lite-open")

print(f"Dataset loaded successfully")
print(f"Available splits: {list(dataset.keys())}")
print(f"Total splits: {len(dataset)}")

Dataset loaded successfully
Available splits: ['af', 'ar', 'bn', 'cs', 'de', 'en', 'es', 'fr', 'hi', 'hu', 'id', 'it', 'ja', 'ko', 'mr', 'ne', 'pt', 'ru', 'sr', 'sw', 'te', 'th', 'uk', 'ur', 'vi', 'wo', 'yo', 'zh', 'zu']
Total splits: 29


## Inspect Dataset Structure

In [4]:
# Show sample data from English split
en_split = dataset["en"]
print(f"Columns: {en_split.column_names}")
print(f"Number of questions in English split: {len(en_split)}")
print("\nSample data:")
print(en_split[0])

Columns: ['question_id', 'question', 'answer', 'cot_content', 'category', 'src']
Number of questions in English split: 470

Sample data:
{'question_id': 72, 'question': 'Determine the number of men needed to build a boat in 77 days if it takes 36 men 132 days to build one.', 'answer': '62 men', 'cot_content': '', 'category': 'business', 'src': 'stemez-Business'}


## Data Preprocessing Function

In [19]:
def preprocess_split_data(split_data):
    """
    Preprocess dataset split by removing empty cot_content column
    and converting to list of dictionaries for Elluminate upload
    """
    # Convert to pandas DataFrame for easier manipulation
    df = split_data.to_pandas()

    # Remove the empty cot_content column
    if "cot_content" in df.columns:
        df = df.drop("cot_content", axis=1)
        print(f"Removed empty cot_content column")

    # Convert to list of dictionaries
    records = df.astype(str).to_dict("records")

    print(f"Preprocessed {len(records)} records")
    print(f"Remaining columns: {list(df.columns)}")

    return records


# Test preprocessing with English split
en_records = preprocess_split_data(dataset["en"])
print(f"\nSample preprocessed record:")
print(en_records[0])

Removed empty cot_content column
Preprocessed 470 records
Remaining columns: ['question_id', 'question', 'answer', 'category', 'src']

Sample preprocessed record:
{'question_id': '72', 'question': 'Determine the number of men needed to build a boat in 77 days if it takes 36 men 132 days to build one.', 'answer': '62 men', 'category': 'business', 'src': 'stemez-Business'}


## Create Collections for Each Language

In [20]:
# Language names mapping for better collection descriptions
language_names = {
    "af": "Afrikaans",
    "ar": "Arabic",
    "bn": "Bengali",
    "cs": "Czech",
    "de": "German",
    "en": "English",
    "es": "Spanish",
    "fr": "French",
    "hi": "Hindi",
    "hu": "Hungarian",
    "id": "Indonesian",
    "it": "Italian",
    "ja": "Japanese",
    "ko": "Korean",
    "mr": "Marathi",
    "ne": "Nepali",
    "pt": "Portuguese",
    "ru": "Russian",
    "sr": "Serbian",
    "sw": "Swahili",
    "te": "Telugu",
    "th": "Thai",
    "uk": "Ukrainian",
    "ur": "Urdu",
    "vi": "Vietnamese",
    "wo": "Wolof",
    "yo": "Yoruba",
    "zh": "Chinese",
    "zu": "Zulu",
}

# Dictionary to store created collections
collections = {}

print("Creating collections for each language...")
for lang_code in tqdm(dataset.keys(), desc="Creating collections"):
    language_name = language_names.get(lang_code, lang_code.upper())
    collection_name = f"MMLU-ProX-Lite-Open-{lang_code.upper()}"
    description = f"MMLU-ProX-Lite open-ended questions in {language_name} ({lang_code}). Contains {len(dataset[lang_code])} questions across multiple academic categories."

    try:
        try:
            collection, created = client.collections.get_or_create(name=collection_name)
            client.collections.delete(collection)
            print(f"Deleted existing collection {collection_name}")
        except Exception as e:
            pass

        collection, created = client.collections.get_or_create(
            name=collection_name,
            description=description,
            variables=preprocess_split_data(dataset[lang_code]),
        )

        collections[lang_code] = collection
        status = "created" if created else "found existing"
        print(
            f"✓ {language_name} ({lang_code}): {status} collection '{collection_name}'"
        )

    except Exception as e:
        print(
            f"✗ Error creating collection for {language_name} ({lang_code}): {str(e)}"
        )

print(f"\nSuccessfully created/found {len(collections)} collections")

Creating collections for each language...


Creating collections:   0%|          | 0/29 [00:00<?, ?it/s]

Deleted existing collection MMLU-ProX-Lite-Open-AF
Removed empty cot_content column
Preprocessed 470 records
Remaining columns: ['question_id', 'question', 'answer', 'category', 'src']


Creating collections:   3%|▎         | 1/29 [00:01<00:35,  1.28s/it]

✓ Afrikaans (af): created collection 'MMLU-ProX-Lite-Open-AF'
Deleted existing collection MMLU-ProX-Lite-Open-AR
Removed empty cot_content column
Preprocessed 470 records
Remaining columns: ['question_id', 'question', 'answer', 'category', 'src']


...

✓ Yoruba (yo): created collection 'MMLU-ProX-Lite-Open-YO'
Deleted existing collection MMLU-ProX-Lite-Open-ZH
Removed empty cot_content column
Preprocessed 470 records
Remaining columns: ['question_id', 'question', 'answer', 'category', 'src']


Creating collections:  97%|█████████▋| 28/29 [00:28<00:01,  1.00s/it]

✓ Chinese (zh): created collection 'MMLU-ProX-Lite-Open-ZH'
Deleted existing collection MMLU-ProX-Lite-Open-ZU
Removed empty cot_content column
Preprocessed 470 records
Remaining columns: ['question_id', 'question', 'answer', 'category', 'src']


Creating collections: 100%|██████████| 29/29 [00:29<00:00,  1.01s/it]

✓ Zulu (zu): created collection 'MMLU-ProX-Lite-Open-ZU'

Successfully created/found 29 collections


## Collection Summary

In [9]:
# Display summary of created collections
print("Collection Summary:")
print("=" * 50)

for lang_code, collection in collections.items():
    language_name = language_names.get(lang_code, lang_code.upper())
    num_questions = len(dataset[lang_code])
    print(
        f"{language_name:12} ({lang_code}): {num_questions:3} questions - Collection ID: {collection.id}"
    )

total_questions = sum(len(dataset[lang_code]) for lang_code in collections.keys())
print("=" * 50)
print(f"Total: {len(collections)} collections, {total_questions:,} questions")

Collection Summary:
Afrikaans    (af): 470 questions - Collection ID: 463
Arabic       (ar): 470 questions - Collection ID: 464
Bengali      (bn): 470 questions - Collection ID: 465
Czech        (cs): 470 questions - Collection ID: 466
German       (de): 470 questions - Collection ID: 467
English      (en): 470 questions - Collection ID: 468
Spanish      (es): 470 questions - Collection ID: 469
French       (fr): 470 questions - Collection ID: 470
Hindi        (hi): 470 questions - Collection ID: 471
Hungarian    (hu): 470 questions - Collection ID: 472
Indonesian   (id): 470 questions - Collection ID: 473
Italian      (it): 470 questions - Collection ID: 474
Japanese     (ja): 470 questions - Collection ID: 475
Korean       (ko): 470 questions - Collection ID: 476
Marathi      (mr): 470 questions - Collection ID: 477
Nepali       (ne): 470 questions - Collection ID: 478
Portuguese   (pt): 470 questions - Collection ID: 479
Russian      (ru): 470 questions - Collection ID: 480
Serbian 